# Pythonによるデモ

本稿ではPythonによるシミュレーターの使い方を示す。実際にはPythonのみで書かれた `simulator.py` ではなく、C++拡張ライブラリを用いる。前者は後者に比べて3倍以上遅いので、実装を確認する程度にすることを勧める。

## 準備

### ダウンロード

リポジトリのページの "Code" --> "Download ZIP" から圧縮ファイルをダウンロードし、それを解凍する。あるいは、`git clone https://github.com/Wandao123/ising_model.git` コマンドを実行する。

|![downloading](downloading.png)|
|:-:|

### Vanilla Python

Pythonの処理系は[Windows Store](https://www.microsoft.com/ja-jp/p/python-38/9mssztt1n39l?activetab=pivot:overviewtab)や[公式ページ](https://www.python.org/)からダウンロード・インストールできる。ここでは仮想環境を作成し、そこへpipを用いて必要なライブラリをインストールする。PowerShellあるいはコマンドプロンプトを起動した上で、次のコマンドを実行する。

```PowerShell
PS> cd ダウンロードしたフォルダ
PS> cd python
PS> python -m venv env
PS> env/Scripts/activate
PS> pip install numpy matplotlib multiprocess jupyterlab
```

### Anaconda

一般的な用途ではなく、データ分析や機械学習などに特化してPythonを用いる場合は[Anaconda](https://www.anaconda.com/)を利用する方法もある。Anacondaを利用する場合、必要なライブラリが既にインストールされている筈なので、前節の手順はほぼ不要である。ただ、独自ライブラリのインストールの関係上、仮想環境の作成自体は勧める。Anaconda PowerShell Promptを起動して、次を実行する。

```PowerShell
PS> conda create -n ising_model
PS> conda activate ising_model
PS> conda install pip
PS> conda install jupyterlab  # Jupyter Labを使う場合。
```

### C++拡張ライブラリのインストール

リポジトリのページの "Releases"（「ダウンロード」節の画像を参照）からリリースのページへ移動する。そこで最新版のwheelパッケージをダウンロードする。

|![downloading-whl](downloading-whl.png)|
|:-:|

ダウンロードが完了したら、pipを用いてそのwheelパッケージをインストールする（次の手順では `env/Scripts/activate` あるいは `conda activate ising_model` で既に仮想環境が有向化されているとする）：

```PowerShell
PS> cd wheelパッケージをダウンロードしたフォルダ
PS> pip install simulatorWithCpp-*-*-*-*.whl
```

ただし、\* の部分には適当なバージョン名やアーキテクチャ名が入る。

## デモンストレーション

必要なライブラリの読み込みを行う。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('../python')
import simulator                      # Pythonのライブラリを使う場合。
#import simulatorWithCpp as simulator  # C++拡張ライブラリを使う場合。

### 頂点と辺の指定方法

グラフ $G=(V, E)$ について、それぞれの集合が
\begin{equation*}
    V = \{\,0, 1\,\}, \quad E = \{\,\{0, 1\}\,\}
\end{equation*}
で与えられたとする。このとき、Hamiltonian関数は
\begin{equation*}
    H(\sigma) = 3 \sigma_0 \sigma_1 - \sigma_0 -2 \sigma_1
\end{equation*}
となる。これを表すコードは次のようになる。ただし、`Write` メソッドはクラスの現在の状態を表示し、`Spins` プロパティは現在のスピン配置を辞書型変数として取得する。

In [ ]:
isingModel = simulator.IsingModel({0: 1.e0, 1: 2.e0}, {(0, 1): 3.e0})
isingModel.Write()
print()
print(isingModel.Spins)

一般に、Hamiltonian関数が
\begin{equation*}
    H(\sigma) = -\sum_{\{x, y\}\in E}J_{x, y} \sigma_x \sigma_y - \sum_{x\in V} h_x \sigma_x
\end{equation*}
と表されているとする。このシミュレーターでは、IsingModelクラスに外部磁場の強さ $\{h_x\}_{x\in V}$ とスピン-スピン結合係数 $\{J_{x, y}\}_{\{x,y\}\in E}$ とをPythonの辞書型変数の形式で渡す。外部磁場の強さは文字列あるいは整数をキーとする辞書である。また、スピン-スピン結合係数は文字列あるいは整数のタプルをキーとする辞書である。このタプルの要素は2つのみであり、かつ `(a, b)` に対して `a < b` でなければならない。何も指定しない場合はそこでの外部磁場の強さやスピン-スピン結合係数が $0$ であるものと解釈される。特に、空の辞書 `{}` を渡した場合は全ての頂点あるいは辺で $0$ になる。

**例**&nbsp;(Erdős-Rényiランダムグラフ)&nbsp;辺の生成をプログラムに任せることで、ランダムグラフやスピングラスが生成できる。実行する度にスピン-スピン結合係数の行列が変化することを確認してみよ。

In [ ]:
import random

maxNodes = 8
probability = 0.5e0
quadratic = {(i, j): -1 if random.random() <= probability else 0 for i in range(maxNodes) for j in range(i + 1, maxNodes)}
isingModel = simulator.IsingModel({}, quadratic)
isingModel.Write()

### アニーリング

`IsingModel` クラスは `Temperature` プロパティを持つ。その値に応じて、`Update` メソッドによるスピンの更新確率が変化する。ここでは、更新アルゴリズムにGlauber力学を指定した上で、各モンテカルロ・ステップ $n$ で温度を下げてゆく。アニーリング・スケジュールには
\begin{equation*}
    T_n = \frac{T_0}{2 \log (n + 1) + 1}, \quad T_0 = 100
\end{equation*}
を用いる。

In [ ]:
# 初期化。
isingModel = simulator.IsingModel({0: 1.e0, 1: 2.e0}, {(0, 1): 3.e0})
isingModel.Algorithm = simulator.Algorithms.Glauber  # 更新アルゴリズムを指定。
isingModel.Write()
T0 = 10.e0
isingModel.Temperature = T0  # 温度の設定。

# サンプリング。
#samples = np.empty((0, 3), dtype=np.float)  # Numpyを使う場合。若干複雑になるので、最後にPythonのリストを変換する方法も併記している。
samples = []
for n in range(5000):
    isingModel.Temperature = T0 / (np.log(1 + n) + 1.e0)
    isingModel.Update()
    #samples = np.append(samples, np.array([n, isingModel.Energy, isingModel.Temperature], dtype=np.float).reshape((1, 3)), axis=0)
    samples.append([n, isingModel.Energy, isingModel.Temperature])

# データの表示。
np.set_printoptions(threshold=30)  # 6行のみ表示。
output = np.array(samples, dtype=np.float)
print(output)

最後に出力されたデータの各列はそれぞれ、ステップ数、エネルギー、温度を表す。横軸をステップ数、縦軸をエネルギーとすると、次のグラフを得る。

In [ ]:
x = output[:, 0]  # 0列目を抽出。
y = output[:, 1]

fig = plt.figure(figsize=(5, 5), dpi=200)
ax = fig.add_subplot(111)
ax.plot(x, y)

同様のことをSCAで行うと次のようになる。

In [ ]:
# 初期化。
isingModel.Algorithm = simulator.Algorithms.SCA
isingModel.Temperature = T0
isingModel.PinningParameter = 3

# サンプリング。
samples = np.empty((0, 3), dtype=np.float)
for n in range(5000):
    isingModel.Temperature = T0 / (np.log(1 + n) + 1.e0)
    isingModel.Update()
    samples = np.append(samples, np.array([n, isingModel.Energy, isingModel.Temperature], dtype=np.float).reshape((1, 3)), axis=0)

# データの表示。
np.set_printoptions(threshold=30)  # 6行のみ表示。
print(samples)

# グラフの描画。
x = samples[:, 0]
y = samples[:, 1]
fig = plt.figure(figsize=(5, 5), dpi=200)
ax = fig.add_subplot(111)
ax.plot(x, y)